In [1]:
from jetcam.usb_camera import USBCamera
import torch
import torchvision.models as models
import ipywidgets
import torch.nn.functional as F
import torch
import torchvision.transforms as transforms
import PIL.Image
import numpy as np
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import threading
import Jetson.GPIO as GPIO
import json
import time
import random
import string
import cv2

WARNNIG: Jetson.GPIO library has not been verified with this carrier board,


In [2]:
# GPIO setup
red_channel = 19
green_channel = 29
channels = [red_channel, green_channel]
GPIO.setmode(GPIO.BOARD)
GPIO.setup(channels, GPIO.OUT, initial=GPIO.LOW)

In [3]:
# Check device number
!ls -ltrh /dev/video*

crw-rw----+ 1 root video 81, 0 Thg 8 24 16:19 /dev/video0


In [4]:
# for USB Camera (Logitech C270 webcam), uncomment the following line
camera = USBCamera(width=224, height=224, capture_device=0) # confirm the capture_device number

camera.running = True
print("camera created")

camera created


In [5]:
PATH = 'sll-more.pth'

In [5]:
PATH = 'sll-less.pth'

In [6]:
#change cache dir to save space
print(torch.hub.get_dir())
torch.hub.set_dir('/mnt/cache')
model = models.alexnet(pretrained=True)
print('model init')


#change last layer to number of categories (27 for 26 letters + empty)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 27)

#cuda
device = torch.device('cuda')
model = model.to(device)
model.load_state_dict(torch.load(PATH))
print('model loaded')

/home/dc19/.cache/torch/hub
model init
model loaded


In [7]:
# image preprocess and labels

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

def preprocess(image):
    global device
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

LABELS = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'empty']


In [8]:
def ordinal_suffix_of(i):
    j = i // 10,
    k = i // 100;
    if j == 1 and k != 1:
        return str(i) + "st"
    if j == 2 and k != 12:
        return str(i) + "nd"
    if j == 3 and k != 13:
        return str(i) + "rd"
    return str(i) + "th"

In [18]:
# game
mode_widget = ipywidgets.ToggleButtons(options=['Spelling', 'Guessing', 'Random Guessing'], description='Game Mode', value='Spelling')
rule_widget = ipywidgets.Textarea(description='Rule', disabled=True, value="""'Current Word' will display a word from dictionary
You will need to spell each letter of that word using the ASL Alphabet
You will have 15 seconds (+2s for each time word get longer) for each word
Wrong spelling will end the game
The word will get longer as time goes on""")
current_word_widget = ipywidgets.Text(description='Current Word', placeholder='Selected word goes here')
answer_widget = ipywidgets.Text(description='Answer', placeholder='Your answer', disabled=True)
timer_widget = ipywidgets.IntProgress(description='Timer', placeholder='Countdown', min=0, max=15)
prediction_widget = ipywidgets.Text(description='Prediction', placeholder='Detected letter goes here')
score_widget = ipywidgets.IntText(description='Score', placeholder='Your score', disabled=True)
state_widget = ipywidgets.ToggleButtons(options=['Start', 'Stop'], description='Game State', value='Stop')

capture_button = ipywidgets.Button(description="Capture Word", disabled=True)
undo_button = ipywidgets.Button(description="Undo Word", disabled=True)
finish_button = ipywidgets.Button(description="Skip Timer", disabled=True)

camera_widget = ipywidgets.Image()

current_index = 0

def live(state_widget, model, camera, prediction_widget, score_widget, mode_widget, timer_widget, current_word_widget, answer_widget):
    global LABELS, current_index
    can_start = False
    while state_widget.value == 'Start':
        image = camera.value
        preprocessed = preprocess(image)
        output = model(preprocessed)
        probs = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
        category_index = probs.argmax()
        prediction_widget.value = LABELS[category_index]
        if can_start and mode_widget.value ==  'Spelling':
            #logic for spelling game
            if len(answer_widget.value) < len(current_word_widget.value) and current_index < len(current_word_widget.value) and LABELS[category_index] == current_word_widget.value[current_index]:
                answer_widget.value += LABELS[category_index]
                current_index += 1
            if len(answer_widget.value) == len(current_word_widget.value):
                timer_widget.value = 0
        if can_start == False:
            #start the game after the prediction is init
            can_start = True
            game_thread = threading.Thread(target=game, args=(score_widget, mode_widget, timer_widget, state_widget, current_word_widget, answer_widget))
            game_thread.start()
        #time.sleep(0.1) # sleep to make the prediction slower

def game(score_widget, mode_widget, timer_widget, state_widget, current_word_widget, answer_widget):
    global current_index
    start_diff = 0 # start with 3 letter words
    end_diff = 7 # end with 10 letter words
    level_up = 3 # increase length every 3 words
    memorize_timer = 10 # timer for memorize word
    answer_timer_max = 30 # max timer to spelling
    selected_word = ''
    
    random.seed(time.time() * 1000)
    
    f = open('words.json')
    words = json.load(f)

    if mode_widget.value == 'Spelling':
        level = start_diff * level_up
        while state_widget.value == 'Start':
            word_set = level // level_up
            timer_widget.max = (word_set * 2) + 15
            if timer_widget.max > answer_timer_max:
                timer_widget.max = answer_timer_max
            if word_set > end_diff:
                word_set = end_diff
            word_list = words[str(word_set)]
            current_word_widget.value = word_list[random.randrange(0, len(word_list))]
            current_index = 0
            timer_widget.value = timer_widget.max 
            while timer_widget.value > 0 and state_widget.value == 'Start':
                time.sleep(1)
                timer_widget.value -= 1
            if current_word_widget.value == answer_widget.value: 
                score_widget.value += 1
                level += 1
                answer_widget.value = ''
                GPIO.output(channels, (GPIO.LOW, GPIO.HIGH))
                time.sleep(1)
                GPIO.output(channels, (GPIO.LOW, GPIO.LOW))
            else:
                current_word_widget.value = 'Game Over'
                state_widget.value = "Stop"
                current_index = 0
                GPIO.output(channels, (GPIO.HIGH, GPIO.LOW))
                time.sleep(1)
                GPIO.output(channels, (GPIO.LOW, GPIO.LOW))
                break
    elif mode_widget.value == 'Guessing':
        start_diff = 2 # start with 5 letter words
        end_diff = 7 # end with 10 letter words
        level_up = 5 # increase length every 5 words
        timer_widget.max = memorize_timer
        timer_widget.value = memorize_timer
        level = start_diff * level_up
        capture_button.disabled = False
        undo_button.disabled = False
        finish_button.disabled = False
        while state_widget.value == 'Start':
            word_set = level // level_up
            if word_set > end_diff:
                word_set = end_diff
            word_list = words[str(word_set)]
            current_word_widget.value = word_list[random.randrange(0, len(word_list))]
            timer_widget.value = timer_widget.max
            capture_button.disabled = True
            undo_button.disabled = True
            answer_widget.value = 'Memorize the word'
            while timer_widget.value > 0 and state_widget.value == 'Start':
                time.sleep(1)
                timer_widget.value -= 1
            capture_button.disabled = False
            undo_button.disabled = False
            answer_widget.value = ''
            current_word = current_word_widget.value
            random_index = random.randrange(0, len(current_word))
            current_word_widget.value = "Guess the " + ordinal_suffix_of(random_index + 1) + " letter"
            timer_widget.value = timer_widget.max
            while timer_widget.value > 0 and state_widget.value == 'Start':
                time.sleep(1)
                timer_widget.value -= 1
            if current_word[random_index] == answer_widget.value:
                score_widget.value += 1
                level += 1
                GPIO.output(channels, (GPIO.LOW, GPIO.HIGH))
                time.sleep(1)
                GPIO.output(channels, (GPIO.LOW, GPIO.LOW))
                answer_widget.value = ''
            else:
                current_word_widget.value = 'Game Over'
                capture_button.disabled = True
                undo_button.disabled = True
                finish_button.disabled = True
                state_widget.value = "Stop"
                GPIO.output(channels, (GPIO.HIGH, GPIO.LOW))
                time.sleep(1)
                GPIO.output(channels, (GPIO.LOW, GPIO.LOW))
                break
    elif mode_widget.value == 'Random Guessing':
        # use start and end diff as str length
        start_diff = 5 # start with 5 letter words
        end_diff = 25 # end with 10 letter words
        timer_widget.max = memorize_timer
        timer_widget.value = memorize_timer
        level = start_diff * level_up
        letters = string.ascii_lowercase
        capture_button.disabled = False
        undo_button.disabled = False
        finish_button.disabled = False
        while state_widget.value == 'Start':
            word_length = level // level_up
            current_word_widget.value = ''.join(random.choice(letters) for i in range(word_length))
            timer_widget.value = timer_widget.max
            capture_button.disabled = True
            undo_button.disabled = True
            answer_widget.value = 'Memorize the word'
            while timer_widget.value > 0 and state_widget.value == 'Start':
                time.sleep(1)
                timer_widget.value -= 1
            capture_button.disabled = False
            undo_button.disabled = False
            answer_widget.value = ''
            current_word = current_word_widget.value
            random_index = random.randrange(0, len(current_word))
            current_word_widget.value = "Guess the " + ordinal_suffix_of(random_index + 1) + " letter"
            timer_widget.value = timer_widget.max 
            while timer_widget.value > 0 and state_widget.value == 'Start':
                time.sleep(1)
                timer_widget.value -= 1
            if current_word[random_index] == answer_widget.value: 
                score_widget.value += 1
                GPIO.output(channels, (GPIO.LOW, GPIO.HIGH))
                time.sleep(1)
                GPIO.output(channels, (GPIO.LOW, GPIO.LOW))
                level += 1
                answer_widget.value = ''
            else:
                current_word_widget.value = 'Game Over'
                capture_button.disabled = True
                undo_button.disabled = True
                finish_button.disabled = True
                state_widget.value = "Stop"
                GPIO.output(channels, (GPIO.HIGH, GPIO.LOW))
                time.sleep(1)
                GPIO.output(channels, (GPIO.LOW, GPIO.LOW))
                break
            
            
def start_game(change):
    # first time starting model will take a while before model can start predicting
    if change['new'] == 'Start':
        mode_widget.disabled = True
        prediction_widget.value="Starting..."
        current_word_widget.value = ''
        timer_widget.value = 0
        score_widget.value = 0
        answer_widget.value = ''
        prediction_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget, score_widget, mode_widget, timer_widget, current_word_widget, answer_widget))
        prediction_thread.start()
    else:
        mode_widget.disabled = False
        
def change_game_mode(change):
    global rule_widget, current_word_widget, timer_widget, prediction_widget, score_widget
    if change['new'] == 'Spelling':
        rule_widget.value = """'Current Word' will display a word from dictionary
You will need to spell each letter of that word using the ASL Alphabet
You will have 15 seconds (+2s for each time word get longer) for each word
Wrong spelling will end the game
The word will get longer as time goes on"""
    elif change['new'] == 'Guessing':
        rule_widget.value = """'Current Word' will display a word from dictionary
You will have 10 seconds to memorize the word
After that 'Current Word' will display the position of the letter you need to guess and you will have 10 seconds to guess the letter
Wrong guess will end the game
The word will get longer as time goes on
'Skip Timer' button can be use to skip the timer"""
    elif change['new'] == 'Random Guessing':
        rule_widget.value = """'Current Word' will display a random string of letters
You will have 10 seconds to memorize the string
After that 'Current Word' will display the position of the letter you need to guess and you will have 10 seconds to guess the letter
Wrong guess will end the game
The word will get longer as time goes on
'Skip Timer' button can be use to skip the timer"""
    current_word_widget.value = ''
    timer_widget.value = 0
    prediction_widget.value = ''
    score_widget.value = 0
    answer_widget.value = ''


font = cv2.FONT_HERSHEY_SIMPLEX
color = (0, 255, 0) 
org = (50, 50)
thickness = 2
fontScale = 1

def draw_label(change):
    image = change['new']
    image = cv2.putText(image, prediction_widget.value.capitalize(), org, font,  
                       fontScale, color, thickness, cv2.LINE_AA)
    camera_widget.value = bgr8_to_jpeg(image)
    
    
def on_capture(b):
   answer_widget.value += prediction_widget.value[0]

def on_undo(b):
   answer_widget.value = answer_widget.value[:-1]

def on_finish(b):
   timer_widget.value = 0 # put timer at 0, use to skip timer
        
mode_widget.observe(change_game_mode, names='value')
state_widget.observe(start_game, names='value')

capture_button.on_click(on_capture)
undo_button.on_click(on_undo)
finish_button.on_click(on_finish)

live_execution_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, ipywidgets.VBox([
        prediction_widget, 
        current_word_widget,
        answer_widget,
        timer_widget,
        score_widget,
        ipywidgets.HBox([undo_button, capture_button]),
        finish_button
    ])]),
    mode_widget,
    rule_widget,
    state_widget
])

print("widgets created")

widgets created


In [19]:
# link camera
camera.unobserve_all()
camera.observe(draw_label, names='value')

In [20]:
all_widget = ipywidgets.VBox([
    ipywidgets.HBox([live_execution_widget])
])

display(all_widget)

In [ ]:
import os
camera.unobserve_all()
camera.running = False
GPIO.cleanup()
os._exit(00)